In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using Microsoft.Extensions.Configuration;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

In [2]:
#r "nuget: Microsoft.Extensions.DependencyInjection, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.11.0"

using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName,
        endpoint: azureOpenAIEndpoint,
        apiKey: azureOpenAIAPIKey)
    .Build();

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();


Installed Packages Microsoft.Extensions.DependencyInjection, 8.0.0 Microsoft.SemanticKernel, 1.11.0

In [3]:
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2500, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var simpleDecisionPrompt = "Provide some decision frameworks that can help improve the quality of decisions.";
var chatResult = await chatCompletionService.GetChatMessageContentAsync(simpleDecisionPrompt, openAIPromptExecutionSettings);

Console.WriteLine(chatResult.Content);

1. SWOT Analysis: This framework helps in evaluating the Strengths, Weaknesses, Opportunities, and Threats related to a decision. It provides a comprehensive view of both internal and external factors that can impact the decision.

2. Decision Matrix: This framework involves creating a matrix to evaluate different options based on specific criteria. Each option is scored based on how well it meets each criterion, and the option with the highest score is selected.

3. Cost-Benefit Analysis: This framework involves comparing the costs and benefits of each option. It helps in identifying the option that provides the maximum benefit for the minimum cost.

4. Pareto Analysis: This framework, also known as the 80/20 rule, helps in identifying the few critical factors that have the most significant impact on the decision. It helps in prioritizing actions based on their potential impact.

5. Decision Tree: This framework involves mapping out different options and their potential outcomes in a 

In [4]:
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(simpleDecisionPrompt))
{
    Console.Write(content);
}

1. SWOT Analysis: Assessing strengths, weaknesses, opportunities, and threats can help in making informed decisions by analyzing internal and external factors that might impact outcomes.

2. Decision Matrix: This involves creating a table with different options and criteria to evaluate each option's performance based on a scoring system. It helps compare various choices and select the best based on data-driven analysis.

3. Cost-Benefit Analysis: Analyzing the costs and benefits of each option can help determine which decision provides the maximum value or return on investment.

4. Pareto Analysis: Also known as the 80/20 rule, this technique helps prioritize tasks or decisions that will provide the most significant impact or results.

5. Decision Tree: This is a graphical representation of decisions and their possible consequences and helps visualize potential outcomes and make informed choices based on probabilities and costs.

6. Six Thinking Hats: This technique involves examining 

In [8]:
// private void WriteMessages(IReadOnlyList<ChatMessageContent> messages, ChatHistory? history = null)
// {
//     foreach (var message in messages)
//     {
//         Console.WriteLine($"# {message.Role}: {message.Content}");
//     }

//     history?.AddRange(messages);
// }

var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";
var simpleDecisionPrompt = "Provide five decision frameworks that can help improve the quality of decisions.";

var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(simpleDecisionPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);


// WriteMessages(await chatCompletionService.GetChatMessageContentsAsync(chatHistory, openAIPromptExecutionSettings), chatHistory);

1. **Cost-Benefit Analysis (CBA)**:
   - **Framework**: Cost-benefit analysis is an approach to making decisions that involves comparing the costs and benefits of various options. This often involves enumerating all potential costs and benefits, quantifying them in monetary terms when possible, and then comparing to see which option offers the greatest net benefit.
   - **Application**: Use this framework when the decision involves significant financial investments, resource allocation, or when the outcomes can be quantified economically. Example scenarios include business investments, policy decisions, or personal finance decisions.

2. **SWOT Analysis**:
   - **Framework**: SWOT stands for Strengths, Weaknesses, Opportunities, and Threats. This analysis helps in understanding both internal and external factors affecting a decision. Strengths and weaknesses are typically internal aspects, while opportunities and threats are external.
   - **Application**: Useful for strategic business

In [9]:
var simpleDecisionPromptFollowup = "Which of these 5 decision frameworks would work best for making a decision to purchase a car. Select only the single best one.";

chatHistory.AddUserMessage(simpleDecisionPromptFollowup);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

// WriteMessages(await chatCompletionService.GetChatMessageContentsAsync(chatHistory, openAIPromptExecutionSettings), chatHistory);

To determine the best decision framework for purchasing a car, let's briefly review what each framework offers in the context of this scenario:

1. **Cost-Benefit Analysis** – This involves listing all the costs associated with purchasing a car (such as price, insurance, maintenance, fuel expenses, etc.) and all the benefits (such as convenience, safety, comfort, etc.). It quantitatively evaluates whether the benefits outweigh the costs.

2. **SWOT Analysis** – This framework assesses the strengths, weaknesses, opportunities, and threats related to a decision. For a car purchase, it considers factors like the vehicle’s reliability (strength), cost of parts (weakness), resale value (opportunity), and market volatility (threat).

3. **Decision Matrix** – Also known as a grid analysis, this method involves setting up a matrix to compare different options based on various factors that are scored and weighted according to importance. For a car, factors might include cost, fuel efficiency, r

In [10]:
var simpleDecisionPromptFollowupPartTwo = "Which of these 5 decision frameworks listed above are used by the military intelligence community?";

chatHistory.AddUserMessage(simpleDecisionPromptFollowupPartTwo);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

// WriteMessages(await chatCompletionService.GetChatMessageContentsAsync(chatHistory, openAIPromptExecutionSettings), chatHistory);

The military intelligence community often deals with complex and high-stakes decisions that require rigorous methodologies to mitigate risks and ensure optimal outcomes. Among the five decision frameworks listed earlier, the following are particularly pertinent for use within the military intelligence community:

1. **Cost-Benefit Analysis**:
   - **Usage**: This framework helps in assessing the feasibility and desirability of various actions by comparing the costs (resources, risks, manpower, etc.) against the benefits (strategic advantage, security enhancement, mission success, etc.). It aids in resource allocation and strategic planning, particularly in operations and procurement.

2. **SWOT Analysis**:
   - **Usage**: SWOT analysis is utilized to evaluate the strategic positioning of a military operation or intelligence strategy. It helps in understanding the strengths (what advantages do we have), weaknesses (what areas need improvement), opportunities (potential favorable conditi

In [13]:
Console.WriteLine("Number of chat interactions: " + chatHistory.Count());

foreach (var message in chatHistory)
{
    Console.WriteLine($"# {message.Role}: {message.Content}");
}

Number of chat interactions: 6
# system: You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
# user: Provide five decision frameworks that can help improve the quality of decisions.
# user: Which of these 5 decision frameworks would work best for making a decision to purchase a car. Select only the single best one.
# assistant: To determine the best decision framework for purchasing a car, let's briefly review what each framework offers in the context of this scenario:

1. **Cost-Benefit Analysis** – This involves listing all the costs associated with purchasing a car (such as price, insurance, maintenance, fuel expenses, etc.) and all the benefits (such as convenience, safety, comfort, etc.). It quantitatively evaluates whether the benefits outweigh the costs.

2. **SWOT Analysis** – This framework asses